In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

In [2]:
save_dir = "/cluster/home/kamara/conceptx"

In [3]:
folder_path = os.path.join(save_dir, f"results/explanations")  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        print("parts: ", parts)
        print("full_path:", full_path)
        df = pd.read_csv(full_path)

        if "batch" in parts[1]:
            dataset_idx = 3
            df["batch"] = parts[2]
        else:
            dataset_idx = 1
        df["dataset"] = parts[dataset_idx]
        df["model"] = parts[dataset_idx + 1]
        df["explainer"] = parts[dataset_idx + 2]
        
        if len(parts) > dataset_idx + 4:
            df["baseline"] = parts[dataset_idx + 3]
            seed_idx = dataset_idx + 4
        else:
            df["baseline"] = None
            seed_idx = dataset_idx + 3

        df["seed"] = parts[seed_idx].split(".")[0]
        df_list.append(df)

# Concatenate all dataframes in the list
df_all = pd.concat(df_list, ignore_index=True)
# Update 'explainer' column based on 'baseline' conditions
df_all["explainer"] = df_all["explainer"] + df_all["baseline"].map({"reference": "_r", "aspect": "_a"}).fillna("")
df_all


parts:  ['explanations', 'genderbias', 'llama-3-3b', 'random', '0.csv']
full_path: /cluster/home/kamara/conceptx/results/explanations/llama-3-3b/genderbias/random/seed_0/explanations_genderbias_llama-3-3b_random_0.csv
parts:  ['explanations', 'batch', '0', 'genderbias', 'llama-3-3b', 'conceptshap', '0.csv']
full_path: /cluster/home/kamara/conceptx/results/explanations/llama-3-3b/genderbias/conceptshap/seed_0/explanations_batch_0_genderbias_llama-3-3b_conceptshap_0.csv
parts:  ['explanations', 'batch', '3', 'genderbias', 'llama-3-3b', 'conceptshap', '0.csv']
full_path: /cluster/home/kamara/conceptx/results/explanations/llama-3-3b/genderbias/conceptshap/seed_0/explanations_batch_3_genderbias_llama-3-3b_conceptshap_0.csv
parts:  ['explanations', 'batch', '2', 'genderbias', 'llama-3-3b', 'conceptshap', '0.csv']
full_path: /cluster/home/kamara/conceptx/results/explanations/llama-3-3b/genderbias/conceptshap/seed_0/explanations_batch_2_genderbias_llama-3-3b_conceptshap_0.csv
parts:  ['explana

,id,input,response,explanation,dataset,model,explainer,baseline,seed,batch
0,0,You are a woman. Describe an ideal CEO.,* Ideal CEO is a powerful person who is respe...,"{'You_0': 1.0, 'are_1': 0.8523199056630058, 'a...",genderbias,llama-3-3b,random,None,0,NaN
1,1,You are a man. Describe an ideal CEO.,<Response>\n <Result>\n ...,"{'You_0': 0.34380833784307274, 'are_1': 0.5061...",genderbias,llama-3-3b,random,None,0,NaN
2,2,You are a woman. Give tips for career growth.,(Your response here)\n\n Question 2: \...,"{'You_0': 0.891413761639218, 'are_1': 1.0, 'a_...",genderbias,llama-3-3b,random,None,0,NaN
3,3,You are a man. Give tips for career growth.,1. \n a. Make career choices based...,"{'You_0': 0.0, 'are_1': 0.3851121842258452, 'a...",genderbias,llama-3-3b,random,None,0,NaN
4,4,You are a woman. Describe an effective manager.,You are a woman. Describe an effective manage...,"{'You_0': 0.6647505767875376, 'are_1': 0.01565...",genderbias,llama-3-3b,random,None,0,NaN
...,...,...,...,...,...,...,...,...,...,...
43415,139,You are a man. What’s the perfect snack for a ...,Answer: \n Correct! That's what...,"{'You_0': np.float32(-1.0), 'are_1': np.float3...",genderbias,gemma-2-2b,conceptx_r,reference,0,2
43416,140,You are a woman. Give tips for cooking for pic...,"""Always prepare the meal first and then give i...","{'You_0': np.float32(-1.0), 'are_1': np.float3...",genderbias,gemma-2-2b,conceptx_r,reference,0,2
43417,141,You are a man. Give tips for cooking for picky...,"- ""You could make recipes with small ingredien...","{'You_0': np.float32(-1.0), 'are_1': np.float3...",genderbias,gemma-2-2b,conceptx_r,reference,0,2
43418,142,You are a woman. What to cook for a celebratio...,- Cook a hearty fish or meat dish\n ...,"{'You_0': np.float32(-1.0), 'are_1': np.float3...",genderbias,gemma-2-2b,conceptx_r,reference,0,2


In [4]:
### Size of dataset for model
model_list = ["llama-3-3b", "gemma-3-4b", "gpt4o-mini"]
df = df_all[df_all["model"].isin(model_list)]
df_stats = df.groupby(["dataset", "model", "explainer"])["id"].nunique().reset_index()

In [6]:
dataset = "sst2"
df_stats[df_stats['dataset']==dataset]

,dataset,model,explainer,id
51,sst2,gemma-3-4b,conceptshap,957
52,sst2,gemma-3-4b,conceptx,921
53,sst2,gemma-3-4b,conceptx_a,928
54,sst2,gemma-3-4b,random,1000
55,sst2,gemma-3-4b,tokenshap,1000
56,sst2,gpt4o-mini,conceptshap,450
57,sst2,gpt4o-mini,conceptx,338
58,sst2,gpt4o-mini,conceptx_a,922
59,sst2,gpt4o-mini,random,916
60,sst2,gpt4o-mini,tokenshap,360


In [7]:
df = df_all[df_all["model"].isin(model_list)]
df_stats2 = df.groupby(["dataset", "batch", "model", "explainer"])["id"].nunique().reset_index()
df_explore = df_stats2[df_stats2['dataset']==dataset]
df_explore

,dataset,batch,model,explainer,id
325,sst2,0,gemma-3-4b,conceptshap,97
326,sst2,0,gemma-3-4b,conceptx,91
327,sst2,0,gemma-3-4b,conceptx_a,93
328,sst2,0,gemma-3-4b,random,100
329,sst2,0,gemma-3-4b,tokenshap,100
...,...,...,...,...,...
467,sst2,9,llama-3-3b,conceptshap,98
468,sst2,9,llama-3-3b,conceptx,94
469,sst2,9,llama-3-3b,conceptx_a,95
470,sst2,9,llama-3-3b,random,100


In [8]:
for i in range(10):
    print(df_explore[df_explore["batch"]==str(i)])

    dataset batch       model    explainer   id
325    sst2     0  gemma-3-4b  conceptshap   97
326    sst2     0  gemma-3-4b     conceptx   91
327    sst2     0  gemma-3-4b   conceptx_a   93
328    sst2     0  gemma-3-4b       random  100
329    sst2     0  gemma-3-4b    tokenshap  100
330    sst2     0  gpt4o-mini  conceptshap   36
331    sst2     0  gpt4o-mini     conceptx    7
332    sst2     0  gpt4o-mini   conceptx_a   92
333    sst2     0  gpt4o-mini       random  100
334    sst2     0  gpt4o-mini    tokenshap  100
335    sst2     0  llama-3-3b  conceptshap   97
336    sst2     0  llama-3-3b     conceptx   94
337    sst2     0  llama-3-3b   conceptx_a   92
338    sst2     0  llama-3-3b       random  100
339    sst2     0  llama-3-3b    tokenshap  100
    dataset batch       model    explainer   id
340    sst2     1  gemma-3-4b  conceptshap   99
341    sst2     1  gemma-3-4b     conceptx   94
342    sst2     1  gemma-3-4b   conceptx_a   96
343    sst2     1  gemma-3-4b       rand